In [1]:
import itertools

In [2]:
class Pair:
    def __init__(self, left, right, depth, parent=None):
        self.left = left
        self.right = right
        self.depth = depth
        self.parent = parent
        
    def magnitude(self):
        left = self.left.magnitude() if isinstance(self.left, Pair) else self.left.value
        right = self.right.magnitude() if isinstance(self.right, Pair) else self.right.value
        return 3 * left + 2 * right
    
    def reduce(self):
        return self.reduce_explode() or self.reduce_split()
    
    def reduce_explode(self):
        if self.depth >= 4:
            return "explode", self
        else:
            return self.left.reduce_explode() or self.right.reduce_explode()
        
    def reduce_split(self):
        return self.left.reduce_split() or self.right.reduce_split()
    
    def __add__(self, other):
        self.increase_depth()
        other.increase_depth()
        new = Pair(self, other, 0)
        self.parent = new
        other.parent = new
        return new
    
    def __repr__(self):
        return f"[{self.left},{self.right}]"
    
    def leaves(self):
        return self.left.leaves() + self.right.leaves()
    
    def increase_depth(self):
        self.depth += 1
        if isinstance(self.left, Pair):
            self.left.increase_depth()
    
        if isinstance(self.right, Pair):
            self.right.increase_depth()

class Number:
    def __init__(self, value, parent=None):
        self.value = value
        self.parent = parent
    
    def magnitude(self):
        return self
    
    def reduce_explode(self):
        return None
    
    def reduce_split(self):
        if self.value >= 10:
            return "split", self
        
        return None
        
    def __add__(self, other):
        self.value += other.value

    def __repr__(self):
        return str(self.value)
        
    def leaves(self):
        return [self]
    
def parse_pair(l: list, depth=0) -> Pair:
    left = Number(l[0]) if isinstance(l[0], int) else parse_pair(l[0], depth + 1)
    right = Number(l[1]) if isinstance(l[1], int) else parse_pair(l[1], depth + 1)
    
    p =  Pair(left, right, depth)
    left.parent = p
    right.parent = p
    return p

In [3]:
def explode(p):
    to_reduce = p
    
    # Find root
    while p.parent:
        p = p.parent
        
    leaves = p.leaves()
    
    left_idx = leaves.index(to_reduce.left)
    if left_idx != 0:
        leaves[left_idx - 1] += to_reduce.left
        
    right_idx = leaves.index(to_reduce.right)
    if right_idx != len(leaves) - 1:
        leaves[right_idx + 1] += to_reduce.right
    
    new = Number(0, parent=to_reduce.parent)
    if to_reduce.parent.left == to_reduce:
        to_reduce.parent.left = new
    else:
        to_reduce.parent.right = new
        
def split(p):
    
    left, right = Number(p.value//2), Number(-(-p.value//2))
    parent = p.parent
    new = Pair(left, right, depth=parent.depth+1, parent=parent)
    left.parent = new
    right.parent = new
    
    if parent.left == p:
        parent.left = new
    else:
        parent.right = new
        
def calc(p):
    while True:
        #print(str(p))
        try:
            action, to_reduce = p.reduce()
            #print(f"{action} {str(to_reduce)}")
        except TypeError:
            break

        if action == "split":
            split(to_reduce)
        elif action == "explode":
            explode(to_reduce)
        
    return p
    

In [4]:
with open("inputs/18", "r") as f:
    lines = f.read().splitlines()

s = parse_pair(eval(lines[0]))
for l in lines[1:]:
    s = s + parse_pair(eval(l))
    calc(s)
    
str(s) == "[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]"
s.magnitude()

3987

In [5]:
max(calc(parse_pair(eval(p1)) + parse_pair(eval(p2))).magnitude() for p1, p2 in itertools.combinations(lines, 2))

4500